In [1]:
from pathlib import Path
import sys  
import os
import dask.dataframe as dd
curr_path = str(Path(os.getcwd()).parent)
sys.path.append(curr_path)
from scripts.constants import *
from scripts.read import *

In [2]:
spark = create_spark()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/13 13:30:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
def read_mapped_industry_data(spark: SparkSession, prefix: str="") -> pd.DataFrame:
    """
    Reads mapped industry data
    - Parameters
        - spark: Start SparkSession
        - prefix: Prefix used when accessing files from other directoriess
    - Returns
        - Pandas consumer dataframe
    """

    df = spark.read.csv(prefix+INDUSTRY_MAPPED, header=True, inferSchema=True)
    pandas_df = df.toPandas()
    return pandas_df

def read_merchant_fraud_probability(spark: SparkSession, prefix: str="") -> pd.DataFrame:
    """
    Reads merchant fraud probability data
    - Parameters
        - spark: Start SparkSession
        - prefix: Prefix used when accessing files from other directoriess
    - Returns
        - Pandas consumer dataframe
    """

    df = spark.read.csv(prefix+CURATED_MERCHANT_FRAUD_PATH, header=True, inferSchema=True)
    pandas_df = df.toPandas()
    return pandas_df

In [4]:
merchants = read_curated_tbl_merchant(spark, '.')
mapped_industry_data = read_mapped_industry_data(spark, prefix=".")
transaction_all = read_curated_transactions_all(spark, prefix=".")
merchant_fraud_probability = read_merchant_fraud_probability(spark, prefix=".")
consumer= read_curated_consumer_external_join(spark, prefix=".")

In [5]:
# only choose the top 3 mapped industry
top3_mapped_industry_data = mapped_industry_data[mapped_industry_data[MAPPED_INDUSTRIES].isin(['X', 'P', 'F'])]
top3_mapped_industry_data

,name,merchant_abn,revenue_level,take_rate,industry_tags,MappedIndustry
5,Fusce Company,10206519221,a,6.34,gift card novelty souvenir,P
19,Ut Consulting,10462560289,c,2.95,gift card novelty souvenir,P
28,Sed Et Libero PC,10651113986,b,3.25,tent awning,X
29,Fermentum Risus Foundation,10702078694,a,5.95,gift card novelty souvenir,P
37,Per Inceptos Company,10901349044,a,5.96,gift card novelty souvenir,P
...,...,...,...,...,...,...
3988,Nunc Ac Incorporated,99053443481,b,4.81,tent awning,X
3990,Odio LLC,99100250331,c,2.05,gift card novelty souvenir,P
3991,Non Enim Corp.,99115883676,b,3.65,digital goods books movies music,F
3992,Non Ante Inc.,99217762645,b,5.05,digital goods books movies music,F


In [6]:
# only keep the column we need in transaction_all
new_transaction_all = transaction_all.select(MERCHANT_ABN, DOLLAR_VALUE, ORDER_ID)

# drop the duplicates in transaction_all
transaction_all_with_no_duplicates = new_transaction_all.dropDuplicates([ORDER_ID])

# transfer to the pandas dataframe
transaction_all_with_no_duplicates_pd = transaction_all_with_no_duplicates.toPandas()

In [7]:
# join the transaction_all with mapped_industry_data on merchant_abn
joined_transaction_and_mapped_industry = pd.merge(transaction_all_with_no_duplicates_pd, mapped_industry_data, on=MERCHANT_ABN, how=INNER_JOIN)

joined_transaction_and_mapped_industry

,merchant_abn,dollar_value,order_id,name,revenue_level,take_rate,industry_tags,MappedIndustry
0,10264435225,79.46,77d9fa32-5336-4453-8a16-6bee5e2ec656,Ipsum Primis Ltd,c,2.39,watch clock jewelry repair,V
1,10264435225,184.72,67550d21-ef53-4de7-bd5d-641a789a886b,Ipsum Primis Ltd,c,2.39,watch clock jewelry repair,V
2,10264435225,38.04,3f435044-cfe8-4012-8959-59f751497ab4,Ipsum Primis Ltd,c,2.39,watch clock jewelry repair,V
3,10264435225,70.71,21f359e6-2994-43c5-8dc1-c5fcdd260fc9,Ipsum Primis Ltd,c,2.39,watch clock jewelry repair,V
4,10264435225,130.87,c028c6a2-e48c-44f0-9cdd-7ea9708c50d3,Ipsum Primis Ltd,c,2.39,watch clock jewelry repair,V
...,...,...,...,...,...,...,...,...
9391907,84787662573,8763.28,abbff7c7-a141-4ddf-9f4b-c209eb97e808,Varius Orci Inc.,c,3.05,antique repairs restoration,R
9391908,39150153670,38089.94,df8d2fa8-3b50-4904-985a-5e2e71cbcf2a,Aliquam Eu Institute,a,6.99,antique repairs restoration,R
9391909,42461534060,22600.11,209f9787-84e4-4fde-89f9-9a5f118912d4,Elit LLP,b,3.98,antique repairs restoration,R
9391910,52524795452,12864.45,8236fc52-c73f-4429-9508-c382578ca6f2,Cras Convallis Associates,a,5.89,antique repairs restoration,R


In [8]:
# join the transaction_all with mapped_industry_data on merchant_abn
joined_transaction_and_mapped_industry = pd.merge(transaction_all_with_no_duplicates_pd, top3_mapped_industry_data, on=MERCHANT_ABN, how=INNER_JOIN)

joined_transaction_and_mapped_industry

,merchant_abn,dollar_value,order_id,name,revenue_level,take_rate,industry_tags,MappedIndustry
0,16388802220,17.74,bc8a39d8-02f8-4d56-aada-bb7f5502f0da,Iaculis Aliquet Institute,a,5.72,gift card novelty souvenir,P
1,16388802220,109.47,6982f742-f99d-4a1a-9a39-e116b0aae1ec,Iaculis Aliquet Institute,a,5.72,gift card novelty souvenir,P
2,16388802220,32.75,8f96bcbf-d24e-4a19-8e12-aa1366eee33d,Iaculis Aliquet Institute,a,5.72,gift card novelty souvenir,P
3,16388802220,25.97,edda7bbc-dc34-4720-9561-3ae7dab92ade,Iaculis Aliquet Institute,a,5.72,gift card novelty souvenir,P
4,16388802220,49.45,ccc1ffd8-dec1-42bf-9fc5-9461838ef03a,Iaculis Aliquet Institute,a,5.72,gift card novelty souvenir,P
...,...,...,...,...,...,...,...,...
3268674,61569154734,1138.15,418f77dc-c476-4543-b7e6-8065d6ef1c14,Blandit At Nisi PC,a,5.78,tent awning,X
3268675,61569154734,1870.59,5b62998d-adb4-45a7-865b-84c82a6d3d78,Blandit At Nisi PC,a,5.78,tent awning,X
3268676,61569154734,622.06,3bb7f1c0-43b1-4438-be67-5ec772d494aa,Blandit At Nisi PC,a,5.78,tent awning,X
3268677,61569154734,696.76,f02ad7e2-243f-4a62-8651-7b475ea64f93,Blandit At Nisi PC,a,5.78,tent awning,X


In [9]:
# group by the name and merchant_abn calculate the average dollar value
average_dollar_value = joined_transaction_and_mapped_industry.groupby([NAME, MERCHANT_ABN])[DOLLAR_VALUE].mean().reset_index()

# group by the name and merchant_abn calculate the median dollar value
median_dollar_value = joined_transaction_and_mapped_industry.groupby([NAME, MERCHANT_ABN])[DOLLAR_VALUE].median().reset_index()

# merge and sort the average and median dollar value
average_and_median_dollar_value = pd.merge(average_dollar_value, median_dollar_value, on=[NAME, MERCHANT_ABN], how=INNER_JOIN, suffixes=(AVERAGE_SUFFIX, MEDIAN_SUFFIX))
# sort the average dollar value
average_and_median_dollar_value = average_and_median_dollar_value.sort_values(by=[DOLLAR_VALUE+AVERAGE_SUFFIX], ascending=False)

# around the dollar_value_avg and dollar_value_median to 3 decimal places
average_and_median_dollar_value[DOLLAR_VALUE+AVERAGE_SUFFIX] = average_and_median_dollar_value[DOLLAR_VALUE+AVERAGE_SUFFIX].round(3)
average_and_median_dollar_value[DOLLAR_VALUE+MEDIAN_SUFFIX] = average_and_median_dollar_value[DOLLAR_VALUE+MEDIAN_SUFFIX].round(3)

average_and_median_dollar_value

,name,merchant_abn,dollar_value_avg,dollar_value_median
454,Sem Pellentesque Ut Corp.,89249209432,1582.582,1430.500
308,Morbi Neque Associates,88935026131,1472.410,1284.535
404,Posuere At Velit Incorporated,62787106107,1443.584,1294.005
399,Porttitor Corp.,40361137142,1404.070,1091.505
481,Tellus Phasellus Industries,62607401721,1372.901,1251.775
...,...,...,...,...
127,Egestas Rhoncus Proin Inc.,61481587663,13.371,9.480
511,Urna Nullam Lobortis Institute,88475230162,13.216,9.340
274,Malesuada Fringilla Est Institute,58154486337,13.127,8.960
154,Eros Limited,47086412084,11.253,8.900


In [10]:
# save the result to csv
CURATED_TOP3_AVERAGE_AND_MEDIAN_DOLLAR_PER_TAG_PATH = f"{TABLES_DIR}top3_average_and_median_dollar_per_tag{CSV}"
PATH = CURATED_TOP3_AVERAGE_AND_MEDIAN_DOLLAR_PER_TAG_PATH

average_and_median_dollar_value.to_csv('.'+ PATH, index=False)

In [11]:
# group by industry_tags and MappedIndustry, and count the number of order_id in each group
number_of_transactions = joined_transaction_and_mapped_industry.groupby([NAME, MERCHANT_ABN])[ORDER_ID].count().reset_index()

# rename the column and sort by number_of_transactions
number_of_transactions.rename(columns={ORDER_ID: NUM_TRANSACTIONS}, inplace=True)
number_of_transactions = number_of_transactions.sort_values(by=[NUM_TRANSACTIONS], ascending=False)

number_of_transactions

,name,merchant_abn,600
382,Pede Nonummy Corp.,64203420245,179799
340,Non Vestibulum Industries,49891706470,170731
238,Lacus Consulting,45629217853,157454
157,Est Nunc Consulting,89726005175,148876
227,Ipsum Dolor Sit Corporation,80324045558,135879
...,...,...,...
399,Porttitor Corp.,40361137142,12
35,Amet Faucibus Incorporated,38249941422,11
71,Blandit At Nisi PC,61569154734,9
475,Suspendisse Eleifend Consulting,34339412732,9


In [12]:
# save the result to csv
CURATED_TOP3_NUMBER_OF_TRANSACTIONS_PATH = f"{TABLES_DIR}top3_number_of_transactions{CSV}"
PATH = CURATED_TOP3_NUMBER_OF_TRANSACTIONS_PATH

number_of_transactions.to_csv('.'+ PATH, index=False)

In [13]:
# merge transaction data with fraud prob
fraud_prob_and_industry = pd.merge(joined_transaction_and_mapped_industry, merchant_fraud_probability, on=MERCHANT_ABN, how=INNER_JOIN)

# calculate the average fraud probability for each industry_tags and MappedIndustry
average_fraud_prob = fraud_prob_and_industry.groupby([NAME, MERCHANT_ABN])[FRAUD_PROBABILITY].mean().reset_index()

# calculate the median fraud probability for each industry_tags and MappedIndustry
median_fraud_prob = fraud_prob_and_industry.groupby([NAME, MERCHANT_ABN])[FRAUD_PROBABILITY].median().reset_index()

# merge and sort
average_and_median_fraud_prob = pd.merge(average_fraud_prob, median_fraud_prob, on=[NAME, MERCHANT_ABN], suffixes=(AVERAGE_SUFFIX, MEDIAN_SUFFIX))
average_and_median_fraud_prob = average_and_median_fraud_prob.sort_values(by=[FRAUD_PROBABILITY+AVERAGE_SUFFIX], ascending=False)

# around the fraud_probability_avg and fraud_probability_median to 3 decimal places
average_and_median_fraud_prob[FRAUD_PROBABILITY+AVERAGE_SUFFIX] = average_and_median_fraud_prob[FRAUD_PROBABILITY+AVERAGE_SUFFIX].round(3)
average_and_median_fraud_prob[FRAUD_PROBABILITY+MEDIAN_SUFFIX] = average_and_median_fraud_prob[FRAUD_PROBABILITY+MEDIAN_SUFFIX].round(3)

average_and_median_fraud_prob

KeyError: 'merchant_abn'

23/10/13 15:29:53 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 151214 ms exceeds timeout 120000 ms
23/10/13 15:29:53 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/13 15:29:58 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at sc

In [ ]:
# save the result to csv
CURATED_TOP3_AVERAGE_AND_MEDIAN_FRAUD_PROBABILITY_PATH = f"{TABLES_DIR}top3_average_and_median_fraud_probability{CSV}"
PATH = CURATED_TOP3_AVERAGE_AND_MEDIAN_FRAUD_PROBABILITY_PATH

average_and_median_fraud_prob.to_csv('.'+ PATH, index=False)

In [ ]:
# seelct the columns we need
earning_columns = [col for col in consumer.columns if any(keyword in col for keyword in ['consumer_id', 'ratio', 'earning', 'earin', 'earner'])]

# transfer to the pandas dataframe
consumer_earning = consumer[earning_columns]
consumer_earning_pd = consumer_earning.toPandas()

# change the data type
consumer_earning_pd[RATIO] = consumer_earning_pd[RATIO].astype(float)
columns_to_convert = consumer_earning_pd.columns.difference([CUSTOMER_ID, RATIO])
consumer_earning_pd[columns_to_convert] = consumer_earning_pd[columns_to_convert].astype(int)

# get the real earning information for each consumer
consumer_earning_pd = consumer_earning_pd.assign(**{col: consumer_earning_pd[col] * consumer_earning_pd[RATIO] for col in consumer_earning_pd.columns if col not in ['consumer_id', RATIO]})
consumer_earning_pd = consumer_earning_pd.drop(columns=[RATIO])
consumer_earning_pd = consumer_earning_pd.groupby([CONSUMER_ID]).mean().reset_index()

In [ ]:
# only keep the column we need in transaction_all also drop the duplicates in transaction_all
transaction_all_with_consumer_id = transaction_all.select(MERCHANT_ABN, CONSUMER_ID, ORDER_ID)
transaction_all_with_consumer_id_with_no_duplicates = transaction_all_with_consumer_id.dropDuplicates([ORDER_ID])

# transfer to the pandas dataframe
transaction_all_with_consumer_id_with_no_duplicates_pd = transaction_all_with_consumer_id_with_no_duplicates.toPandas()

In [ ]:
# merge the data
consumer_earning_and_transaction = pd.merge(consumer_earning_pd, transaction_all_with_consumer_id_with_no_duplicates_pd, on=CONSUMER_ID, how=INNER_JOIN)
consumer_earning_and_transaction_and_industry = pd.merge(consumer_earning_and_transaction, top3_mapped_industry_data, on=MERCHANT_ABN, how=INNER_JOIN)
earning_columns = columns_to_convert.drop(CONSUMER_ID)

# group by the industry_tags and MappedIndustry, and calculate the average earning for each group
average_earning = consumer_earning_and_transaction_and_industry.groupby([NAME, MERCHANT_ABN])[earning_columns].mean().reset_index()
average_earning[earning_columns] = average_earning[earning_columns].round(3)

average_earning

In [ ]:
# save the result to csv
CURATED_TOP3_AVERAGE_EARNING_PATH = f"{TABLES_DIR}top3_average_earning{CSV}"
PATH = CURATED_TOP3_AVERAGE_EARNING_PATH

average_earning.to_csv('.'+ PATH, index=False)